# Notebook 00: Baselines and Limitations

## Understanding Why Classical ML Fails for Nuclear Data

**Learning Objective:** Understand *why* classical machine learning fails for nuclear data evaluation using real experimental data.

**Focus Isotopes:**
- **U-235 Fission** (data-rich, well-understood): Critical for nuclear reactors
- **Cl-35 (n,p)** (data-sparse, research interest): Important for astrophysics and medical applications

### The Problem

Nuclear cross sections σ(E) are smooth, continuous functions of energy. They exhibit:
- **Resonance peaks**: Sharp but smooth features (especially visible in U-235)
- **Threshold behavior**: σ(E) = 0 for E < E_threshold, then rises smoothly
- **Physical constraints**: Conservation laws, unitarity, causality

### Why This Matters

A reactor calculation uses millions of cross-section evaluations. If predictions are:
- **Jagged** → Unphysical neutron transport
- **Discontinuous** → Numerical instabilities
- **Wrong at key energies** → Incorrect k_eff (criticality)

This is the **Validation Paradox**: Low MSE ≠ Safe Reactor!

**Additional Challenge:** How do models perform when data is sparse (like Cl-35)?

---

## Part 1: The Naive Approach

Let's examine why tree-based models struggle with real nuclear cross-section data from IAEA EXFOR.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from nucml_next.data import NucmlDataset
from nucml_next.baselines import XGBoostEvaluator, DecisionTreeEvaluator

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Verify EXFOR data exists
exfor_path = Path('../data/exfor_processed.parquet')
if not exfor_path.exists():
    raise FileNotFoundError(
        f"EXFOR data not found at {exfor_path}\n"
        "Please run: python scripts/ingest_exfor.py --exfor-root <path> --output data/exfor_processed.parquet"
    )

print("✓ Imports successful")
print("✓ EXFOR data found")
print("Welcome to NUCML-Next: Understanding ML Limitations with Real Nuclear Data")

### Step 1.1: Load Real EXFOR Data (Tabular View)

We'll train on the **full EXFOR database** (all isotopes), then evaluate on U-235 and Cl-35. This is the correct ML approach:
- **Training**: Learn general nuclear physics patterns from ALL available data
- **Evaluation**: Test predictions on specific target isotopes (U-235, Cl-35)

This demonstrates true transfer learning and generalization, not just memorization!

In [ ]:
# CRITICAL FIX: Train on ENTIRE EXFOR database, not just U-235 and Cl-35!
# Otherwise we're just memorizing the test set (overfitting)

# Load FULL dataset for training (NO isotope filters)
print("Loading ENTIRE EXFOR database for training...")
dataset_full = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='tabular',
    # No Z/A filters = train on all available isotopes
    filters={
        'MT': [18, 102, 103]  # Focus on fission, capture, (n,p) reactions
    }
)

# Project to tabular format with NAIVE features
df_naive = dataset_full.to_tabular(mode='naive')

print(f"✓ Full training dataset: {df_naive.shape}")
print(f"\nFeatures (Naive Mode):")
print(df_naive.columns.tolist())

# Show isotope distribution in training data
print("\n📊 Training Data Distribution (Top 10 Isotopes):")
isotope_counts = dataset_full.df.groupby(['Z', 'A']).size().sort_values(ascending=False).head(10)
for (z, a), count in isotope_counts.items():
    # Simple element lookup (extend as needed)
    element_map = {92: 'U', 17: 'Cl', 94: 'Pu', 26: 'Fe', 8: 'O', 1: 'H',
                   82: 'Pb', 6: 'C', 13: 'Al', 7: 'N', 11: 'Na', 79: 'Au'}
    elem = element_map.get(z, f'Z{z}')
    print(f"  {elem}-{a:3d}: {count:>8,} measurements")

print(f"\n✓ Total isotopes in training: {len(isotope_counts)} unique Z/A combinations")
print(f"✓ This allows the model to learn general nuclear physics patterns!")

# Now load our evaluation targets (U-235 and Cl-35)
print("\nLoading evaluation targets (U-235 and Cl-35)...")
dataset_eval = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='tabular',
    filters={
        'Z': [92, 17],     # Uranium and Chlorine
        'A': [235, 35],    # U-235 and Cl-35
        'MT': [18, 102, 103]
    }
)

print(f"✓ Evaluation dataset: {len(dataset_eval.df)} measurements")
print("\n📊 Evaluation Isotopes:")
for (z, a), group in dataset_eval.df.groupby(['Z', 'A']):
    isotope = f"{'U' if z==92 else 'Cl'}-{a}"
    print(f"  {isotope:8s}: {len(group):>6,} measurements")

df_naive.head()

**Notice:** The naive approach treats reactions as independent categories (MT_2, MT_18, etc.).

**Problem:** This ignores physics! (n,2n) and (n,3n) are related - they differ by one neutron.

But tree-based models don't know this. To them, MT=16 and MT=17 are just labels.

### Step 1.2: Train Decision Tree (The "Villain")

We'll intentionally configure the tree to show the **staircase effect**.

In [ ]:
# Initialize Decision Tree with limited depth (exaggerates stairs)
dt_model = DecisionTreeEvaluator(
    max_depth=6,          # Shallow tree = coarse stairs
    min_samples_leaf=20,  # Large leaves = big steps
)

# Train on naive features
dt_metrics = dt_model.train(df_naive)

print("\n" + "="*60)
print("Decision Tree Performance:")
print("="*60)
for key, value in dt_metrics.items():
    print(f"  {key:20s}: {value}")

### Step 1.3: The Failure Mode - Visualize the Staircase Effect

Let's predict cross sections for both isotopes and see what happens...

**U-235**: Rich data → Can the model capture resonances?  
**Cl-35**: Sparse data → Can the model interpolate gaps?

In [ ]:
# Create comparative visualization: U-235 (data-rich) vs Cl-35 (data-sparse)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# LEFT: U-235 fission in resonance region
Z_u, A_u = 92, 235
mt_u = 18  # Fission
energy_range_u = (1.0, 100.0)  # 1-100 eV (resonance region)

# Get U-235 ground truth from evaluation dataset
mask_u = (dataset_eval.df['Z'] == Z_u) & (dataset_eval.df['A'] == A_u) & (dataset_eval.df['MT'] == mt_u)
df_truth_u = dataset_eval.df[mask_u].copy()
df_truth_u = df_truth_u[(df_truth_u['Energy'] >= energy_range_u[0]) & 
                         (df_truth_u['Energy'] <= energy_range_u[1])]

if len(df_truth_u) > 0:
    # Get Decision Tree predictions (dense sampling to see steps)
    energies_dt_u, predictions_dt_u = dt_model.predict_resonance_region(
        Z_u, A_u, mt_u, energy_range_u, num_points=500, mode='naive'
    )
    
    # Plot U-235
    ax1.plot(df_truth_u['Energy'], df_truth_u['CrossSection'], 
            'b-', linewidth=2.5, label='Ground Truth (EXFOR)', alpha=0.7, zorder=1)
    ax1.plot(energies_dt_u, predictions_dt_u, 
            'r-', linewidth=1.5, label='Decision Tree (Staircase)', alpha=0.8, zorder=2)
    
    ax1.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
    ax1.set_title('U-235 Fission (DATA-RICH): Staircase Effect\n' + 
                  f'{len(df_truth_u)} EXFOR measurements in range\n' +
                  f'(Model trained on full EXFOR database)',
                  fontsize=13, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.set_yscale('log')
    ax1.grid(True, alpha=0.3)
    
    # Annotate the problem
    ax1.annotate('Unphysical steps!\n(Real resonances are smooth)',
                xy=(30, predictions_dt_u[150]), xytext=(50, predictions_dt_u[150]*3),
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                fontsize=10, color='red', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
else:
    ax1.text(0.5, 0.5, 'No U-235 fission data in range\n(Check EXFOR ingestion)',
             ha='center', va='center', transform=ax1.transAxes, fontsize=11)
    ax1.set_title('U-235 Fission (No Data)', fontsize=13, fontweight='bold')

# RIGHT: Cl-35 (n,p) reaction
Z_cl, A_cl = 17, 35
mt_cl = 103  # (n,p)
energy_range_cl = (1e6, 2e7)  # 1-20 MeV (fast neutron region)

# Get Cl-35 ground truth from evaluation dataset
mask_cl = (dataset_eval.df['Z'] == Z_cl) & (dataset_eval.df['A'] == A_cl) & (dataset_eval.df['MT'] == mt_cl)
df_truth_cl = dataset_eval.df[mask_cl].copy()
df_truth_cl = df_truth_cl[(df_truth_cl['Energy'] >= energy_range_cl[0]) & 
                           (df_truth_cl['Energy'] <= energy_range_cl[1])]

if len(df_truth_cl) > 0:
    # Get Decision Tree predictions
    energies_dt_cl, predictions_dt_cl = dt_model.predict_resonance_region(
        Z_cl, A_cl, mt_cl, energy_range_cl, num_points=500, mode='naive'
    )
    
    # Plot Cl-35
    ax2.scatter(df_truth_cl['Energy'], df_truth_cl['CrossSection'], 
               s=80, c='blue', marker='o', label=f'Ground Truth ({len(df_truth_cl)} EXFOR pts)', 
               alpha=0.7, zorder=2, edgecolors='black', linewidths=1)
    ax2.plot(energies_dt_cl, predictions_dt_cl, 
            'r-', linewidth=1.5, label='Decision Tree (Extrapolation)', alpha=0.8, zorder=1)
    
    ax2.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
    ax2.set_title('Cl-35 (n,p) (DATA-SPARSE): Transfer Learning Test\n' + 
                  f'Only {len(df_truth_cl)} EXFOR measurements!\n' +
                  f'(Model learned from other isotopes)',
                  fontsize=13, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3)
    
    # Annotate the challenge
    ax2.annotate('Can the model\ntransfer knowledge?',
                xy=(energies_dt_cl[250], predictions_dt_cl[250]), 
                xytext=(energies_dt_cl[350], predictions_dt_cl[250]*0.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                fontsize=10, color='red', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
else:
    ax2.text(0.5, 0.5, 'No Cl-35 (n,p) data in range\n(Check EXFOR ingestion or expand --max-files)',
             ha='center', va='center', transform=ax2.transAxes, fontsize=11)
    ax2.set_title('Cl-35 (n,p) (No Data)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️  OBSERVATIONS:")
print("="*80)
print("Training Approach: Model trained on FULL EXFOR database (all isotopes)")
print("Evaluation: Testing predictions on U-235 and Cl-35")
print()
print("LEFT (U-235 - Data-Rich in training):")
print("  • Decision Tree creates JAGGED predictions even with lots of training data")
print("  • Staircase effect would cause numerical instabilities in reactor codes")
print()
print("RIGHT (Cl-35 - Data-Sparse, transfer learning):")
print("  • Model must transfer knowledge from other isotopes")
print("  • Large gaps between measurements → Predictions test generalization")
print("  • This is where physics-informed models REALLY shine!")
print("="*80)

### 🔴 Critical Insight #1: Piecewise Constant ≠ Physics

Decision trees partition feature space into rectangles:
```
if Energy < 10.5:
    if Energy < 5.2:
        return 150.0  # Constant!
    else:
        return 89.0   # Jump!
else:
    return 45.0
```

Real physics:
```
σ(E) = σ_0 * Γ / ((E - E_r)² + Γ²/4)  # Smooth Breit-Wigner!
```

---

## Part 2: Can XGBoost Save Us?

Let's try a more sophisticated ensemble method.

In [ ]:
# Initialize XGBoost
xgb_naive = XGBoostEvaluator(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
)

# Train on naive features
xgb_metrics_naive = xgb_naive.train(df_naive)

print("\n" + "="*60)
print("XGBoost Performance (Naive Features):")
print("="*60)
for key, value in xgb_metrics_naive.items():
    if value is not None:
        print(f"  {key:20s}: {value}")

In [ ]:
# Get XGBoost predictions for U-235 (data-rich example)
Z, A, mt_code = 92, 235, 18  # U-235 fission
energy_range = (1.0, 100.0)  # Resonance region

energies_xgb, predictions_xgb = xgb_naive.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000, mode='naive'
)

# Get ground truth from evaluation dataset
mask = (dataset_eval.df['Z'] == Z) & (dataset_eval.df['A'] == A) & (dataset_eval.df['MT'] == mt_code)
df_truth = dataset_eval.df[mask].copy()
df_truth = df_truth[(df_truth['Energy'] >= energy_range[0]) & (df_truth['Energy'] <= energy_range[1])]

# Get Decision Tree predictions (from earlier)
energies_dt, predictions_dt = dt_model.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000, mode='naive'
)

# Comparative plot
fig, ax = plt.subplots(figsize=(12, 6))

# Ground truth
ax.plot(df_truth['Energy'], df_truth['CrossSection'], 
        'b-', linewidth=3, label='Ground Truth (EXFOR)', alpha=0.7, zorder=1)

# Decision Tree (stairs)
ax.plot(energies_dt, predictions_dt, 
        'r--', linewidth=1.5, label='Decision Tree (Staircase)', alpha=0.6, zorder=2)

# XGBoost (smoother but not smooth)
ax.plot(energies_xgb, predictions_xgb, 
        'g-', linewidth=2, label='XGBoost (Better, but...)', alpha=0.8, zorder=3)

ax.set_xlabel('Energy (eV)', fontsize=12, fontweight='bold')
ax.set_ylabel('Cross Section (barns)', fontsize=12, fontweight='bold')
ax.set_title('XGBoost vs Decision Tree: Improvement but Still Not Physics-Compliant\nU-235 Fission (Model trained on full EXFOR)',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ XGBoost is SMOOTHER (ensemble averaging)")
print("✗ But still has micro-steps and can't guarantee smoothness")
print("✗ No awareness of resonance physics")

### 🟡 Critical Insight #2: Ensembles Help, But...

XGBoost averages many trees, which smooths predictions.

**BUT:**
- Still piecewise constant at fine scale
- No guarantee of smoothness
- Can't learn resonance physics (Breit-Wigner shape)
- Poor extrapolation beyond training data

---

## Part 3: The Upgrade - Physics-Aware Features

What if we give XGBoost *better features*?

Instead of naive [Z, A, E, MT_onehot], use physics-derived features from the graph:
- **Q-value**: Reaction energy
- **Threshold**: E_threshold
- **ΔZ, ΔA**: Nuclear topology

This is the bridge to deep learning!

In [ ]:
# Get physics-aware tabular projection from FULL training dataset
df_physics = dataset_full.to_tabular(mode='physics')

print("Physics-Aware Features (trained on full EXFOR):")
print(df_physics.columns.tolist())
print(f"\nDataset shape: {df_physics.shape}")
print(f"\nFirst few rows:")
df_physics.head()

In [ ]:
# Train XGBoost with physics features
xgb_physics = XGBoostEvaluator(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
)

xgb_metrics_physics = xgb_physics.train(df_physics)

print("\n" + "="*60)
print("XGBoost Performance (Physics Features):")
print("="*60)
for key, value in xgb_metrics_physics.items():
    if value is not None:
        print(f"  {key:20s}: {value}")

print("\nComparison with Naive Features:")
print(f"  Test MSE (Naive):   {xgb_metrics_naive['test_mse']:.4e}")
print(f"  Test MSE (Physics): {xgb_metrics_physics['test_mse']:.4e}")
improvement = (xgb_metrics_naive['test_mse'] - xgb_metrics_physics['test_mse']) / xgb_metrics_naive['test_mse'] * 100
print(f"  Improvement: {improvement:.1f}%")

In [ ]:
# Get physics-mode predictions for U-235
energies_xgb_phys, predictions_xgb_phys = xgb_physics.predict_resonance_region(
    Z, A, mt_code, energy_range, num_points=1000, mode='physics'
)

# Final comparison
fig, ax = plt.subplots(figsize=(14, 7))

# Ground truth
ax.plot(df_truth['Energy'], df_truth['CrossSection'], 
        'b-', linewidth=3, label='Ground Truth (EXFOR)', alpha=0.8, zorder=1)

# XGBoost naive
ax.plot(energies_xgb, predictions_xgb, 
        'orange', linewidth=2, linestyle='--', label='XGBoost (Naive Features)', alpha=0.6, zorder=2)

# XGBoost physics
ax.plot(energies_xgb_phys, predictions_xgb_phys, 
        'g-', linewidth=2.5, label='XGBoost (Physics Features)', alpha=0.8, zorder=3)

ax.set_xlabel('Energy (eV)', fontsize=13, fontweight='bold')
ax.set_ylabel('Cross Section (barns)', fontsize=13, fontweight='bold')
ax.set_title('Physics Features Help... But We Can Do Better!\nU-235 Fission Resonance Region (Model trained on full EXFOR)',
             fontsize=15, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Physics features improve accuracy")
print("✓ Model learns about thresholds and reaction energetics")
print("✓ Training on full EXFOR allows transfer learning to specific isotopes")
print("✗ STILL can't guarantee smooth resonance curves")
print("✗ STILL poor extrapolation to unseen energy ranges")
print("✗ No explicit physics constraints (unitarity, conservation laws)")

### 🟢 Critical Insight #3: Features Matter, But Architecture Matters More

Physics-aware features help XGBoost understand reactions better.

**BUT** the fundamental problem remains:
- Tree-based models are **piecewise constant**
- No inductive bias for **smoothness**
- No way to encode **physical constraints**

---

## Part 4: Feature Importance Analysis

Let's see what XGBoost "thinks" is important.

In [ ]:
# Get feature importance
importance_physics = xgb_physics.get_feature_importance()

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(importance_physics['Feature'], importance_physics['Importance'])
ax.set_xlabel('Importance (Gain)', fontsize=12, fontweight='bold')
ax.set_title('XGBoost Feature Importance (Physics Mode)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
print(importance_physics.head())

### 🎓 Key Takeaway

> **Low MSE on test data does NOT guarantee safe reactor predictions!**
>
> We need models that:
> 1. Respect physics (smoothness, thresholds, unitarity)
> 2. Extrapolate correctly (beyond training data)
> 3. Prioritize safety-critical reactions (sensitivity weighting)
> 4. **Handle data-sparse scenarios** (like Cl-35) without overfitting
>
> This is why we need **Physics-Informed Deep Learning**.

---

## Summary: Why Classical ML Fails

| Issue | U-235 (Data-Rich) | Cl-35 (Data-Sparse) |
|-------|-------------------|---------------------|
| Staircase Effect | 🔴 Severe (even with lots of data) | 🔴 Severe |
| Interpolation | 🟡 Approximate | 🔴 Very poor (large gaps) |
| Extrapolation | 🔴 Fails | 🔴 Completely fails |
| Physics Constraints | 🔴 None | 🔴 None |
| Uncertainty Quantification | 🔴 Poor | 🔴 Very poor |
| Training Speed | 🟢 Fast | 🟢 Fast |

### The Path Forward

We need:
1. **Graph Neural Networks** → Learn nuclear topology (not just Z, A)
2. **Transformers** → Learn smooth energy sequences σ(E)
3. **Physics-Informed Loss** → Enforce unitarity, thresholds, conservation
4. **Transfer Learning** → Use U-235 knowledge to improve Cl-35 predictions
5. **Uncertainty Quantification** → Know when to trust sparse-data predictions

---

## Next Steps

In **Notebook 01**, we'll:
- Build the **Chart of Nuclides as a Graph**
- Visualize nuclear topology connecting U-235 and Cl-35
- Understand how GNNs can transfer knowledge between isotopes

In **Notebook 02**, we'll:
- Implement **GNN + Transformer**
- Train on graph-structured real data
- See **smooth, physics-compliant predictions** for both isotopes!

In **Notebook 03**, we'll:
- Integrate with **OpenMC** for U-235 reactor validation
- Achieve reactor-grade accuracy with real nuclear data
- Demonstrate uncertainty quantification for Cl-35

Continue to `01_Data_Fabric_and_Graph.ipynb` →